In [173]:
#single_image = r"C:\Users\danie\Desktop\tiny rainbow.jpg"
image_folder = r"C:\Users\midyr\Downloads\colorwheels"
image_pathway = r"C:\Users\midyr\Downloads\traffic4.jpg"

TOO_SMALL = 20
SMALL = 500000
MEDIUM = 1200000
LARGE = 2100000
    
X_train = []
Y_train = []
counter = 1
    
height = 256
width = 256

total_images = 1

In [174]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import glob
import cv2
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D, InputLayer, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io, color
from skimage.io import imread, imshow, imsave
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [175]:
def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [176]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg') #.png

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    
    print("processing", df_length, "photos for training")
    
    height_list= []
    width_list = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height_list.append(img.shape[0])
        width_list.append(img.shape[1])
        
    df["height"] = height_list
    df["width"] = width_list
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    df.head()
    
    return df

In [177]:
def pca_rgb(imgBGR, size): #imgRGB, size
    b,g,r = cv2.split(imgBGR)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
        
    RGB_compressed = np.float32(RGB_compressed)
    print("PCA SHAPE",np.min(RGB_compressed),np.max(RGB_compressed))    
    return RGB_compressed

In [178]:
def rgb_pad(image):
    
    img_height = image.shape[0]
    print(height)
    img_width = image.shape[1]
    print(width)
    
    WHITE = [255,255,255] #[255,255,255]  1, 1, 1
    
    if img_height % 2 == 0:
        top = int((height - img_height)/ 2)
        bottom = int((height - img_height)/ 2)
    else: 
        top = int((height - (img_height-1))/ 2)
        bottom = int(((height - (img_height-1))/ 2)-1)

    if img_width % 2 == 0:
        left = int((width - img_width)/ 2)
        right = int((width - img_width)/ 2)
    else: 
        left = int((width - (img_width-1))/ 2)
        right = int(((width - (img_width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,value=WHITE)
    print(image.shape)
    print(np.min(image),np.max(image))

    return image

In [179]:
def prep_img(img_path):
    imgRGB = img_to_array(load_img(img_path))
    imgRGB = np.array(imgRGB, dtype=float)
    print('Padded:', np.min(imgRGB), np.max(imgRGB))



    image_size_category = shape(imgRGB) 
    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    if PCA_YN == "yes":
        RGB_compressed = pca_rgb(imgRGB, image_size_category)
        RGB_compressed = np.clip(RGB_compressed,0,1) * 255
        print("PCA ran")
    else:
        RGB_compressed = imgRGB
        print("PCA did not run")


    
    padded_image = rgb_pad(RGB_compressed)

    print('Padded:', np.min(padded_image), np.max(padded_image))
    X_LAB = color.rgb2lab(1.0/255*padded_image)[:,:,0] #1.0/255*padded_image
    print('X_LAB:', np.min(X_LAB), np.max(X_LAB))
    Y_LAB1 = color.rgb2lab(1.0/255*padded_image)[:,:,1:] #1.0/255*padded_image
    print('Y_LAB_org:', np.min(Y_LAB1), np.max(Y_LAB1))
    #Y_LAB = Y_LAB1 / 128
    Y_LAB = (Y_LAB1 + 128)/ 256
    print('Y_LAB:', np.min(Y_LAB), np.max(Y_LAB))


    print(X_LAB.shape)
    X = X_LAB.reshape(1, height, width, 1) #comment out when show predicted
    Y = Y_LAB.reshape(1, height, width, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [180]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

In [181]:
PCA_YN = "no"

#prep_image(single_image)

df = create_dataframe(image_folder)
prep_dataframe(df)

processing 4 photos for training
dropped  0 photos because they were too large to process
dropped  0 photos because they were too small to process
Padded: 0.0 255.0
PCA did not run
256
256
(256, 256, 3)
0.0 255.0
Padded: 0.0 255.0
X_LAB: 0.17702971378012577 100.0
Y_LAB_org: -71.17268893328688 77.75729265802376
Y_LAB: 0.22198168385434813 0.8037394244454052
(256, 256)
image processed
Padded: 0.0 255.0
PCA did not run
256
256
(256, 256, 3)
0.0 255.0
Padded: 0.0 255.0
X_LAB: 0.039573654069084085 100.0
Y_LAB_org: -90.63866915032773 83.20574458892847
Y_LAB: 0.1459426986315323 0.8250224398005018
(256, 256)
image processed
Padded: 0.0 255.0
PCA did not run
256
256
(256, 256, 3)
0.0 255.0
Padded: 0.0 255.0
X_LAB: 0.8013688253736433 100.0
Y_LAB_org: -48.95291303408261 90.26595316207187
Y_LAB: 0.3087776834606148 0.8526013795393432
(256, 256)
image processed
Padded: 0.0 255.0
PCA did not run
256
256
(256, 256, 3)
0.0 255.0
Padded: 0.0 255.0
X_LAB: 0.4324681123001284 99.45842141301871
Y_LAB_org: -3

# MODEL

In [182]:
BATCH_SIZE = 3
LEARNING_RATE = 0.001 #0.001
EPOCHS = 500 #500
KERNEL_SIZE = 2 
NUM_CLASSES = 64 #64
#IMAGE_WIDTH = 128 #128
#IMAGE_HEIGHT = 128 #128
#SIGMA = 5

In [183]:
def mode():
   
    model = tf.keras.Sequential([
    Conv2D(64, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(64, kernel_size=KERNEL_SIZE, activation='relu', padding='same', strides=(2, 2)),
    BatchNormalization(),

    Conv2D(128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(128, kernel_size=KERNEL_SIZE, activation='relu', padding='same', strides=(2, 2)),
    BatchNormalization(),

    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same', strides=(2, 2)),
    BatchNormalization(),

    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    BatchNormalization(),

    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    BatchNormalization(),

    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    Conv2D(512, kernel_size=KERNEL_SIZE, activation='relu', padding='same', dilation_rate=2),
    BatchNormalization(),

    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(256, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    BatchNormalization(),

    UpSampling2D(size=(2, 2)),
    Conv2D(128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    Conv2D(128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'),
    BatchNormalization(),

    Conv2D(64, kernel_size=KERNEL_SIZE, padding='same'),
    Conv2D(64, kernel_size=KERNEL_SIZE, padding='same'),
    Conv2D(64, kernel_size=KERNEL_SIZE, padding='same'),
    BatchNormalization(),
        
    UpSampling2D(size=(2, 2)),
    Conv2D(16, kernel_size=KERNEL_SIZE, padding='same'),    
    Conv2D(16, kernel_size=KERNEL_SIZE, padding='same'),
    Conv2D(16, kernel_size=KERNEL_SIZE, padding='same'),    
    BatchNormalization(), 
        
    Conv2D(2, kernel_size=KERNEL_SIZE, padding='same'),
    Conv2D(2, kernel_size=KERNEL_SIZE, padding='same'),
    Conv2D(2, kernel_size=KERNEL_SIZE, padding='same'),
    BatchNormalization(),
        
    UpSampling2D(size=(2, 2))])
    
    model.compile(optimizer ="rmsprop", loss = "mae", metrics = ["accuracy"])
    
    return model 

#?? Acitivation function and Loss function #CategoricalCrossentropy
#Conv2D(NUM_CLASSES, kernel_size=1, padding='same', activation='softmax'), UpSampling2D(size=(4, 4))])

In [184]:
model = mode()

In [185]:
model.build((1,256,256,1))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_341 (Conv2D)         (1, 256, 256, 64)         320       
                                                                 
 conv2d_342 (Conv2D)         (1, 128, 128, 64)         16448     
                                                                 
 batch_normalization_121 (Ba  (1, 128, 128, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_343 (Conv2D)         (1, 128, 128, 128)        32896     
                                                                 
 conv2d_344 (Conv2D)         (1, 64, 64, 128)          65664     
                                                                 
 batch_normalization_122 (Ba  (1, 64, 64, 128)         512       
 tchNormalization)                                   

In [186]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [187]:
len(X_train)

4

In [ ]:
%%time
history = model.fit(train_data, batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = 2)  # add validation_split = 0.2

Epoch 1/500
4/4 - 4s - loss: 0.8287 - accuracy: 0.4563 - 4s/epoch - 1s/step
Epoch 2/500
4/4 - 2s - loss: 0.6996 - accuracy: 0.4275 - 2s/epoch - 508ms/step
Epoch 3/500
4/4 - 2s - loss: 0.6793 - accuracy: 0.4412 - 2s/epoch - 517ms/step
Epoch 4/500
4/4 - 2s - loss: 0.6250 - accuracy: 0.4104 - 2s/epoch - 554ms/step
Epoch 5/500
4/4 - 2s - loss: 0.6028 - accuracy: 0.4415 - 2s/epoch - 505ms/step
Epoch 6/500
4/4 - 2s - loss: 0.5446 - accuracy: 0.4890 - 2s/epoch - 487ms/step
Epoch 7/500
4/4 - 2s - loss: 0.4992 - accuracy: 0.5019 - 2s/epoch - 491ms/step
Epoch 8/500
4/4 - 2s - loss: 0.4858 - accuracy: 0.5076 - 2s/epoch - 487ms/step
Epoch 9/500
4/4 - 2s - loss: 0.4812 - accuracy: 0.5065 - 2s/epoch - 493ms/step
Epoch 10/500
4/4 - 2s - loss: 0.4769 - accuracy: 0.5025 - 2s/epoch - 493ms/step
Epoch 11/500
4/4 - 2s - loss: 0.4728 - accuracy: 0.4987 - 2s/epoch - 486ms/step
Epoch 12/500
4/4 - 2s - loss: 0.4687 - accuracy: 0.4960 - 2s/epoch - 531ms/step
Epoch 13/500
4/4 - 2s - loss: 0.4647 - accuracy: 0.4

In [ ]:
#filename = 'Singular_Model'
#model2.save(filename)
#saved_model = load_model("Singular Model")

# PREDICTION

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
def get_photo(image):
    prep_color, prep_bw = prep_img(image_pathway)
    test_predict = model.predict(prep_bw)
    #print(test_predict)
    print('test_range_A:', np.min(test_predict[:,:,0]), np.max(test_predict[:,:,0]))
    print('test_range_B:', np.min(test_predict[:,:,1]), np.max(test_predict[:,:,1]))
    
    test_predict_reshaped= test_predict[0,:,:,:]
    #test_predict_reshaped = test_predict_reshaped*128
    test_predict_reshaped = (test_predict_reshaped*256)-128
    #print(test_predict_reshaped)
    BW_reshaped  = prep_bw[0,:,:,:]
    
    print('test_reshaped_A:', np.min(test_predict_reshaped[:,:,0]), np.max(test_predict_reshaped[:,:,0]))
    print(len(np.unique(test_predict_reshaped[:,:,0]))) #, return_counts=True
    print('test_reshaped_B:', np.min(test_predict_reshaped[:,:,1]), np.max(test_predict_reshaped[:,:,1]))
    test_merged_LAB = np.dstack((BW_reshaped, test_predict_reshaped))
    test_merged_rgb = color.lab2rgb(test_merged_LAB)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(test_merged_rgb)

In [ ]:
data = np.load('C:/Study/Semester2/Machine Learning/Github/bin_centers.npy')

In [ ]:
print(data.shape)

In [ ]:
get_photo(image_pathway)